# Segmenting and Clustering Neighborhoods in Toronto

## I. Download and Prepare Dataset

### 1. Import Library

In [24]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from bs4 import BeautifulSoup

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


### 2. Scrape Data

In [81]:
# scrape the Wikipedia page to obtain the raw data
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data=requests.get(url).text
#data

In [82]:
# use BeautifulSoup to get process on web text
soup = BeautifulSoup(data,'html.parser')
#print(soup.prettify())

In [83]:
table = soup.find('table')
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        l.append(row)

In [84]:
# Create dataframe
df = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,None
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


### 3. Preprocess Data

#### Requirements  
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.  
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park.  
- These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.  
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.  
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [85]:
df.describe()

,PostalCode,Borough,Neighbourhood
count,180,180,103
unique,180,11,98
top,M3Z,Not assigned,Downsview
freq,1,77,4


Only process cells that have an assigned Borough. Ignore cells with a borough that is Not assigned.

In [86]:
df=df[df.Borough != 'Not assigned']
df.head(10)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [87]:
df.describe()

,PostalCode,Borough,Neighbourhood
count,103,103,103
unique,103,10,98
top,M1P,North York,Downsview
freq,1,24,4


#### Evaluating for Missing Data
As the output shows: after excluding the rows with cells with a borough that is Not assigned, there is <b>NO</b> Not assigned neighborhood. 

In [88]:
missing_data = df.isnull()
missing_data.head(5)
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")    

PostalCode
False    103
Name: PostalCode, dtype: int64

Borough
False    103
Name: Borough, dtype: int64

Neighbourhood
False    103
Name: Neighbourhood, dtype: int64



In [89]:
df.shape

(103, 3)

The zip code is no longer listed multiple times on the Wiki website. The multiple neighborhoods have already been combined, with a "/". This data cleaning step could be skipped. 

#### Load Coordinates

In [90]:
!pip install geocoder
import geocoder #import geocoder

#!conda install -c conda-forge geopy
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#!conda install -c conda-forge folium=0.5.0 
#import folium # map rendering library

In [109]:
#Load the coordinates from the csv file
df_coordinates = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [110]:
# rename the column "PostalCode"
df_coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Use the csv file to create the new dataframe with coordinates.

In [111]:
df = df.set_index('PostalCode')

KeyError: 'PostalCode'

In [107]:
df_coordinates = df_coordinates.set_index('PostalCode')
df_coordinates.head()

KeyError: 'PostalCode'

In [106]:
df_new = pd.concat([df, df_coordinates.reindex(df.index)], axis=1)
df_new.head()

,Borough,Neighbourhood,Latitude,Longitude
PostalCode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [105]:
df_new = df_new.reset_index()
df_new.head(12)

,level_0,index,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,3,3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,4,4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,5,5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,6,6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,7,7,M3B,North York,Don Mills,43.745906,-79.352188
8,8,8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,9,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
